# Topics

In [57]:
import pandas as pd
from stop_words import get_stop_words
import string
import unicodedata
from num2words import num2words

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

Cargamos los datos obtenidos en el primer punto de la práctica.

In [58]:
df = pd.read_csv('./dataset/amazon.csv')

Recortamos el dataset a los primeros 3000 registros (recuerdo que en el punto anterior ya se mezclaron estos registros).

In [59]:
df = df.iloc[:3000]
df.dropna(inplace=True)

In [60]:
df.isnull().any()

reviewText    False
overall       False
dtype: bool

Podemos eliminar la columna overall porque no aporta información para este ejercicio.

In [61]:
df.drop('overall', axis=1, inplace=True)
df.head()

,reviewText
0,"When I ordered this trap, we had a critter, mo..."
1,This trap was requested to replace an older HA...
2,I waited to write this as it is both difficult...
3,I had purchased one a while back for my electr...
4,received them in four days....just in time for...


Vamos a realizar el preprocesado del texto.

In [62]:
# Obtenemos un diccionario de lemas
lemmas_dict = {}
with open('./dataset/lemmatization-en.txt', 'r', encoding='utf-8') as f:
    for line in f:
        (key, val) = line.split()
        lemmas_dict[str(val)] = key

In [63]:
# Obtenemos una lista de stopwords
sw_list = get_stop_words('en')

In [64]:
# Tabla para eliminar signos de puntuación
table = str.maketrans('', '', string.punctuation)
# Procesamos los textos
processed_texts = []
for text in df['reviewText']:
    processed_text = []
    # Convertimos el texto a minúsuculas
    text = text.lower()
    # Eliminamos caracteres extraños
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # Segmentamos el texto en frases
    sentences = text.split('.')
    # Para cada frase
    for sentence in sentences:
        words = sentence.split(' ')
        # Para cada palabra
        for word in words:
            # Eliminamos stopwords y signos de puntuación
            if word not in string.punctuation and word not in sw_list: 
                word = word.translate(table)
                word = lemmas_dict.get(word, word)
                # Pasamos lo números a palabras
                if word.isdigit():
                    word = num2words(word, lang='en')
                # Eliminamos palabras menores de 3 letras
                if len(word) > 3:
                    processed_text.append(word)
    processed_texts.append(processed_text)

In [65]:
len(processed_texts)

2997

In [66]:
print(processed_texts[5])

['plus', 'minus', 'clip', 'roomy', 'large', 'stem', 'soft', 'little', 'likely', 'bruise', 'otherwise', 'damage', 'stem', 'good', 'thing', 'plant', 'clip', 'however', 'softness', 'tend', 'make', 'seem', 'rather', 'flimsy', 'reserve', 'judgment', 'aspect', 'weather', 'season', 'garden', 'love', 'fact', 'lock', 'pull', 'casually', 'plant', 'buffet', 'weather', 'maraud', 'wildlife', 'though', 'confess', 'clip', 'easy', 'dislodge', 'either', 'possibly', 'need', 'good', 'deal', 'force', 'open', 'however', 'problem', 'order', 'place', 'move', 'hand', 'rather', 'foliage', 'normal', 'huge', 'problem', 'plant', 'rose', 'good', 'bite', 'girl', 'take', 'kindly', 'constrain', 'hang', 'them', 'imagine', 'much', 'wrong', 'sort', 'clip', 'right', 'little', 'skittish', 'thank', 'goodness', 'winter', 'practice', 'think', 'overall', 'like', 'clip', 'necessarily', 'know', 'like', 'much', 'well', 'really', 'issue', 'work', 'work', 'that', 'important', 'thing', 'garden', 'give', 'expensive', 'twist', 'reusa

Como podemos ver ya tenemos los textos de las 3000 reviews y sus palabras más significativas.

Vamos a preparar el texto para para introducirlo a un modelo LDA y entrenarlo.

In [67]:
dictionary = Dictionary(processed_texts)

In [68]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [69]:
# Comprobamos el tamaño de nuestro diccionario
len(dictionary)

11159

In [70]:
# Creamos una lista numérica con el índice de la palabra y su frecuencia en el texto
corpus = [dictionary.doc2bow(doc) for doc in processed_texts]

In [71]:
print(corpus[21])

[(3, 1), (15, 1), (42, 1), (103, 1), (108, 1), (125, 1), (193, 1), (194, 1), (202, 1), (261, 1), (295, 2), (344, 1), (377, 1), (437, 1), (708, 1), (709, 1), (710, 1), (711, 1), (712, 1), (713, 1), (714, 1), (715, 1)]


Entrenamos al modelo LDA.

In [72]:
# Elegimos 3 topics porque hemos mezclado 3 tipos de reviews (bebés, jardín e intrumentos musicales)
num_topics = 3

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    iterations=5,
    passes=10,
    alpha='auto'
)

In [73]:
lda_model.show_topics()

[(0,
  '0.012*"good" + 0.011*"easy" + 0.009*"make" + 0.009*"bottle" + 0.009*"time" + 0.009*"baby" + 0.008*"great" + 0.008*"work" + 0.008*"just" + 0.007*"little"'),
 (1,
  '0.009*"like" + 0.009*"feeder" + 0.009*"good" + 0.008*"work" + 0.007*"will" + 0.006*"product" + 0.006*"little" + 0.006*"mouse" + 0.006*"just" + 0.006*"trap"'),
 (2,
  '0.016*"good" + 0.012*"like" + 0.012*"sound" + 0.011*"guitar" + 0.010*"string" + 0.010*"just" + 0.010*"great" + 0.009*"work" + 0.008*"need" + 0.008*"well"')]

In [74]:
word_dict = {};
for i in range(num_topics):
    words = lda_model.show_topic(i, topn = 20)
    word_dict['Topic #' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic #01,Topic #02,Topic #03
0,good,like,good
1,easy,feeder,like
2,make,good,sound
3,bottle,work,guitar
4,time,will,string
5,baby,product,just
6,great,little,great
7,work,mouse,work
8,just,just,need
9,little,trap,well


In [56]:
# Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # Medida de qué tan bueno es el modelo. Cuanto más bajo, mejor

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.669118913200004

Coherence Score:  0.32918668570312865


In [75]:
# Visualizar los topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.049091  0.074542       1        1  36.653503
1     -0.074242 -0.065053       2        1  33.106480
2      0.123333 -0.009489       3        1  30.240007, topic_info=         Term        Freq       Total Category  logprob  loglift
448     sound  504.000000  504.000000  Default  30.0000  30.0000
430    guitar  501.000000  501.000000  Default  29.0000  29.0000
451    string  450.000000  450.000000  Default  28.0000  28.0000
380    feeder  407.000000  407.000000  Default  27.0000  27.0000
784     pedal  292.000000  292.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
529      want  156.764557  457.227692   Topic3  -5.6115   0.1256
84   conjurer  166.894928  673.586304   Topic3  -5.5489  -0.1992
291     thing  152.163589  481.870300   Topic3  -5.6413   0.0433
55      think  148.544495  544.354065   Topic3  -5.6654  -0.1027
336      come  141.057327  535.309448   Topic3  -5.7171  -0.1377

[257 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
222       1  0.011138  acoustic
222       2  0.011138  acoustic
222       3  0.980106  acoustic
328       1  0.011162     apply
328       2  0.959960     apply
...     ...       ...       ...
62        2  0.303516      work
62        3  0.333025      work
521       1  0.391677      year
521       2  0.357944      year
521       3  0.251123      year

[491 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [77]:
pyLDAvis.save_html(vis, './topics_vis_0.html')

Si abrimos el fichero html que hemos generado y ponemos landa a 0,7 podemos observar:

En el topic 1 la palabra más importante es baby y hay otras muchas palabras que tienen que ver con bebé. Como hija, bote, cambiar, mes, pezón, pequeño, silla... Parece que el topic 1 ha detectado bastante bien la categoría bebé.

El topic 2 parece que se refiere a jardín. Tiene algunas palabras muy significativas, que son ratón, pájaro, comedero, semilla y manguera que prácticamente sólo aparecen en este topic y otras como trampa, jardín, ardilla que tambien son bastante significativas.

El topic 3, por tanto, debe referirse a los instrumentos musicales y así es. Las palabras más significativas son sonido, pedal, cuerda y guitarra. Ademas de otras como cable, grabar, micrófono, tocar. Este toopic está clarisimamente diferenciado.

Por tanto podemos estar satisfechos porque el modelo ha funcionado bastante bien y ha distinguido de una manera bastante clara nuestros tres topics.